In [1]:
import os 
from openai import OpenAI

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
endpoint = "https://models.github.ai/inference"
model_name = "openai/gpt-4o"

client = OpenAI(
    base_url=endpoint,
    api_key=os.environ['GITHUB_API_TOKEN'],
    )

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "What is the capital of England?",
        }
    ],
    temperature=0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name

)



In [5]:
from typing import List
from pydantic import BaseModel, Field

In [6]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [7]:
foo = User(name="John Doe", age=30, email="jondon@gmail.com")

In [8]:
foo.name

'John Doe'

In [9]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [10]:
foo.age

'bar'

In [11]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [12]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [13]:
foo_p.name

'Jane'

In [14]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing

In [15]:
class Class(BaseModel):
    students: List[pUser]

In [16]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [17]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])